# 🔬 Initialization & data wrangling

I denne blok sættes environmentet op, med initialisering af nødvendige libraries samt upload af data. Formålet er at skabe et udgangspunkt for data analyse med python og dets stærke data wrangling toolkits såsom pandas og numpy, data analysis vha. seaborn og kvantitativ analyse hjulpet af machine learning librariet scikit-learn 

In [ ]:
import pandas as pd